In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../../")
import os
from povertymapping.rollout_grids import get_region_filtered_bingtile_grids 
from povertymapping.iso3 import get_iso3_code
import geopandas as gpd

/home/jace/workspace/unicef-ai4d-poverty-mapping/env/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


# Timor Leste Model Rollout Part 2 (Generate Roll-out Grids) 

This notebook generates the rollout grid tiles over the country. The output file is used as an input for Step 3, where we will run the trained model over the set of grids.

The generated grids are set at 2.4km (zoom level 14), matching the grids used during model training. The grids are also filtered to only include populated areas based on Meta's High Resolution Settlement Layer (HRSL) data.

## Set country-specific parameters

In [3]:
REGION = 'timor-leste'
ADMIN_LVL = 'ADM2'
ZOOM_LEVEL = 14
country_code = get_iso3_code(REGION, code='alpha-2').lower()

## Generate Grids

In [4]:
admin_grids_gdf = get_region_filtered_bingtile_grids(
    REGION, 
    admin_lvl=ADMIN_LVL, 
    quadkey_lvl=ZOOM_LEVEL, 
    use_cache=True
)

2023-03-14 12:50:41.581 | INFO     | povertymapping.rollout_grids:get_region_filtered_bingtile_grids:259 - Loading cached grids file /home/jace/.cache/geowrangler/quadkey_grids/timor-leste_14_ADM2_populated_admin_grids.geojson


## Explore Populated Grids

In [5]:
admin_grids_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2024 entries, 0 to 2023
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   quadkey     2024 non-null   object  
 1   shapeName   2024 non-null   object  
 2   shapeISO    2024 non-null   object  
 3   shapeID     2024 non-null   object  
 4   shapeGroup  2024 non-null   object  
 5   shapeType   2024 non-null   object  
 6   pop_count   2024 non-null   float64 
 7   geometry    2024 non-null   geometry
dtypes: float64(1), geometry(1), object(6)
memory usage: 126.6+ KB


In [6]:
admin_grids_gdf.head(2)

,quadkey,shapeName,shapeISO,shapeID,shapeGroup,shapeType,pop_count,geometry
0,31011220203121,Nitibe,None,TLS-ADM2-3_0_0-B58,TLS,ADM2,102.251936,"POLYGON ((124.03564 -9.34067, 124.03564 -9.318..."
1,31011220203123,Nitibe,None,TLS-ADM2-3_0_0-B58,TLS,ADM2,992.492772,"POLYGON ((124.03564 -9.36235, 124.03564 -9.340..."


In [ ]:
admin_grids_gdf.explore()

## Save to file

In [8]:
rollout_date = '-'.join(os.getcwd().split('/')[-2].split('-')[:3])
grid_save_path = f'./{rollout_date}-{country_code}-rollout-grids.geojson'
admin_grids_gdf.to_file(grid_save_path, driver='GeoJSON', index=False)